In [1]:
# repo root
%cd /work

/work


In [2]:
!ls 

LICENSE    adapters  examples  paper	       resources  src
README.md  docker    out       pyproject.toml  scripts	  tests


In [3]:
#!chmod +x resources/gene_id_maps/build_entrez_id_map.sh
#! ./resources/gene_id_maps/build_entrez_id_map.sh

In [4]:
#!chmod +x resources/gene_id_maps/build_ensembl_to_symbol_from_gtf.sh
#! ./resources/gene_id_maps/build_ensembl_to_symbol_from_gtf.sh --download

In [5]:
# Fetch from UCSC Xena
!python paper/scripts/fig2_fetch_inputs.py

[fetch_inputs] OK
  /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz
  /work/paper/source_data/PANCAN_TP53_v1/raw/mc3.v0.2.8.PUBLIC.xena.gz
  /work/paper/source_data/PANCAN_TP53_v1/raw/TCGA_phenotype_dense.tsv.gz


In [6]:
# Extract TP53 mutations (mc3.v0.2.8.PUBLIC.xena.gz)
# Create barcode→cancer from phenotype 
# Output derived/groups/{CANCER}.groups.tsv　for each cancer type

!python paper/scripts/fig2_make_groups.py

[make_groups] sample_type_id raw unique (head): ['07', '06', '05', '03', '02', '01', '11', '']
[make_groups] sample_type_id norm counts: {'1': 10593, '11': 1475, '6': 396, '3': 200, '': 72, '2': 56, '5': 11, '7': 1}
[make_groups] filtered to primary-like sample_type_id in {1,3}: 10793
[make_groups] phenotype mapped samples: 10793
[make_groups] cancers: ['ACC', 'BLCA', 'BRCA', 'CESC', 'CHOL', 'COAD', 'DLBC', 'ESCA', 'GBM', 'HNSC', 'KICH', 'KIRC', 'KIRP', 'LAML', 'LGG', 'LIHC', 'LUAD', 'LUSC', 'MESO', 'OV', 'PAAD', 'PCPG', 'PRAD', 'READ', 'SARC', 'SKCM', 'STAD', 'TGCT', 'THCA', 'THYM', 'UCEC', 'UCS', 'UVM']
[make_groups] MC3 TP53 rows: 3729
[make_groups] MC3 TP53 protein-altering rows (pre-FILTER): 3620
[make_groups] FILTER column not found; not applying FILTER
[make_groups] WARNING: LAML has n_mut=0, n_wt=200
[make_groups] WARNING: UVM has n_mut=0, n_wt=80
[make_groups] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/PANCAN.groups.tsv
  cancers: 33
  samples (mapped): 

In [7]:
!rm -rf /work/paper/source_data/PANCAN_TP53_v1/sample_cards/*
!python paper/scripts/fig2_make_sample_cards.py

[make_sample_cards] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/sample_cards (33 cancers × 2 cards)


In [47]:
!cat /work/paper/source_data/PANCAN_TP53_v1/sample_cards/HNSC.note.sample_card.json

{
  "comparison": "TP53_mut_vs_TP53_wt",
  "disease": "HNSC",
  "extra": {
    "audit_tau": 0.8,
    "benchmark_id": "PANCAN_TP53_v1",
    "context_gate_mode": "note",
    "gene_id_map_tsv": "resources/gene_id_maps/id_map.tsv.gz",
    "goal": "context-conditioned pathway claim selection with calibrated abstention",
    "schema_version": "v1"
  },
  "k_claims": 100,
  "notes": "TCGA Pan-Cancer; groups defined from MC3 TP53 PASS mutations.",
  "perturbation": "genotype",
  "tissue": "tumor"
}


In [48]:
!cat /work/paper/source_data/PANCAN_TP53_v1/sample_cards/HNSC.hard.sample_card.json

{
  "comparison": "TP53_mut_vs_TP53_wt",
  "disease": "HNSC",
  "extra": {
    "audit_tau": 0.8,
    "benchmark_id": "PANCAN_TP53_v1",
    "context_gate_mode": "hard",
    "gene_id_map_tsv": "resources/gene_id_maps/id_map.tsv.gz",
    "goal": "context-conditioned pathway claim selection with calibrated abstention",
    "schema_version": "v1"
  },
  "k_claims": 100,
  "notes": "TCGA Pan-Cancer; groups defined from MC3 TP53 PASS mutations.",
  "perturbation": "genotype",
  "tissue": "tumor"
}


In [6]:
for c in ("HNSC", "LUAD", "LUSC", "BRCA", "OV",  "UCEC",  "SKCM"):
  !Rscript paper/scripts/fig2_deg_rank.R $c


[deg_rank] inputs
  cancer: HNSC 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/HNSC.groups.tsv 
[deg_rank] matched samples: 520  / expr= 11069  / groups= 528 
[deg_rank] QC: dropping zero-variance genes: 269 
[deg_rank] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  n_genes: 20262 
[deg_rank] inputs
  cancer: LUAD 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/LUAD.groups.tsv 
[deg_rank] matched samples: 515  / expr= 11069  / groups= 520 
[deg_rank] QC: dropping zero-variance genes: 338 
[deg_rank] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/LUAD.deg_ranking.tsv 
  n_genes: 20193 
[deg_rank] inputs
  cancer: LUSC 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/gr

In [7]:
!Rscript paper/scripts/fig2_fgsea_to_evidence_table.R HNSC


[fgsea_to_evidence] inputs
  cancer: HNSC 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  out : /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  msigdbr_version: 25.1.1 
  fgsea_version  : 1.36.2 
[fgsea_to_evidence] stats diagnostics
  stats_len: 20262 
  stats_type: double 
  stats_na: 0 
  stats_range: -12.6721379948078,10.5253182402821 
  gene_example: 4060,6817,5153,1902,19594,9055,6447,16500,1970,8301 
  gene_id_type: entrez
 [100%] Downloaded 33730584 bytes...
[fgsea_to_evidence] msigdbr entrez column: ncbi_gene 
[fgsea_to_evidence] overlap diagnostics
  overlap_summary: min 24 p25 80 median 141 p75 165 max 193 
  pathways_with_overlap>=15: 50 / 50 
[fgsea_to_evidence] fgsea diagnostics
  n_pathways: 50 
  res_rows  : 50 
[fgsea_to_evidence] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  n_terms: 50 


In [140]:
# Fig2 run
!rm -rf /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/*
!python paper/scripts/run_fig2_pipeline.py \
  --cancers HNSC \
  --variants ours,context_swap \
  --gate-modes hard \
  --taus 0.2,0.4,0.6,0.8,0.9 \
  --force

[run_fig2] plan
  benchmark_id: PANCAN_TP53_v1
  cancers: HNSC
  variants: ours,context_swap
  gate_modes: hard
  taus: 0.20,0.40,0.60,0.80,0.90
  k_claims: 100
  seed: 42
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out
[run_fig2] jobs: 10
[run_fig2] (1/10) HNSC ours gate=hard tau=0.20
[run_fig2] (2/10) HNSC context_swap gate=hard tau=0.20
[run_fig2] (3/10) HNSC ours gate=hard tau=0.40
[run_fig2] (4/10) HNSC context_swap gate=hard tau=0.40
[run_fig2] (5/10) HNSC ours gate=hard tau=0.60
[run_fig2] (6/10) HNSC context_swap gate=hard tau=0.60
[run_fig2] (7/10) HNSC ours gate=hard tau=0.80
[run_fig2] (8/10) HNSC context_swap gate=hard tau=0.80
[run_fig2] (9/10) HNSC ours gate=hard tau=0.90
[run_fig2] (10/10) HNSC context_swap gate=hard tau=0.90
[run_fig2] OK
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out


In [118]:
!ls /work/paper/source_data/PANCAN_TP53_v1/out/HNSC

context_swap  ours


In [119]:
!ls /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard

tau_0.20  tau_0.40  tau_0.60  tau_0.80	tau_0.90


In [120]:
!cat /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/run_meta.json
!cat /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/sample_card.tau.json


{
  "tool": "llm-pathway-curator",
  "cmd": "run",
  "run_id": "1769125790_557ae5e39f",
  "status": "ok",
  "started_epoch": 1769125790.2579002,
  "config": {
    "evidence_table": "/work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv",
    "sample_card": "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/sample_card.tau.json",
    "outdir": "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80",
    "force": true,
    "seed": 42,
    "run_meta_name": "run_meta.json",
    "tau": 0.8,
    "k_claims": 100
  },
  "env": {
    "python": "3.12.12 | packaged by conda-forge | (main, Oct 22 2025, 23:16:53) [GCC 14.3.0]",
    "platform": "Linux-6.12.54-linuxkit-aarch64-with-glibc2.39",
    "pandas": "2.3.3"
  },
  "inputs": {
    "evidence_table": {
      "path": "/work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv",
      "size_bytes": 12916,
      "mtime_epoch": 1769108252.647574
    },
    "sample_c

In [121]:
p="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv"

import pandas as pd
p = "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv"
df = pd.read_csv(p, sep="\t")
print("shape", df.shape)
print(df["status"].value_counts(dropna=False))
for c in ["abstain_reason","fail_reason","context_status","context_evaluated","context_method","audit_notes"]:
    if c in df.columns:
        print("\n==", c, "==")
        print(df[c].fillna("").value_counts().head(20))

shape (40, 61)
status
PASS       35
ABSTAIN     5
Name: count, dtype: int64

== abstain_reason ==
abstain_reason
            35
unstable     5
Name: count, dtype: int64

== fail_reason ==
fail_reason
    40
Name: count, dtype: int64

== context_status ==
context_status
        35
PASS     5
Name: count, dtype: int64

== context_evaluated ==
context_evaluated
False    35
True      5
Name: count, dtype: int64

== context_method ==
context_method
none     35
proxy     5
Name: count, dtype: int64

== audit_notes ==
audit_notes
ok                                   35
survival[module]=0.750 < tau=0.80     2
survival[module]=0.766 < tau=0.80     1
survival[module]=0.797 < tau=0.80     1
survival[module]=0.672 < tau=0.80     1
Name: count, dtype: int64


In [122]:
import json, pandas as pd

p = "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv"
df = pd.read_csv(p, sep="\t")

def has_keys(s, keys):
    try:
        o = json.loads(str(s))
        return {k: (k in o) for k in keys}
    except Exception:
        return None

keys = ["context_evaluated","context_method","context_status","context_reason","context_confidence","context_notes"]
sample = df[df["abstain_reason"]=="context_missing"].head(5)

for i, r in sample.iterrows():
    ck = has_keys(r.get("claim_json",""), keys)
    print("row", i, "claim_json_has", ck)
    # ついでに中身の該当フィールドだけ見たいなら：
    try:
        o = json.loads(r["claim_json"])
        print({k:o.get(k,None) for k in keys})
    except Exception as e:
        print("json load error:", e)
    print("---")


In [123]:
import pandas as pd
p="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv"
df=pd.read_csv(p, sep="\t")
print("\n".join(df.columns))

claim_id
entity
direction
context_keys
term_uid
source
term_id
term_name
module_id
module_missing
module_reason
module_prefix_effective
gene_ids
term_ids
gene_set_hash
context_score
context_evaluated
eligible
term_survival
keep_term
keep_reason
claim_json
preselect_tau_gate
context_score_proxy
select_notes
select_diag_n_total
select_diag_n_eligible
select_diag_n_ineligible
select_diag_n_picked
select_diag_blocked_by_module_cap
select_diag_k
select_diag_max_per_module
claim_mode
llm_notes
context_method
context_status
context_reason
context_notes
context_confidence
context_gate_mode
status
link_ok
stability_ok
contradiction_ok
stress_ok
stress_evaluated
abstain_reason
fail_reason
audit_notes
tau_used
term_survival_agg
stability_scope
module_id_effective
gene_set_hash_effective
term_ids_set_hash
evidence_key
direction_norm
gene_ids_str
term_ids_str
gene_symbols
gene_symbols_str


In [124]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.20


BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.20
report   True 78607
audit    True 60423
modules  True 14057
edges    True 88967
terms    True 3188
distilled True 40803
risk     True 913
card     True 563
meta     True 5507

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (40, 63)
[report] decisions: {'PASS': 40}
[report] tau unique: [0.2]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'condition', 'disease', 'cancer', 'comparison', 'tau', 'species', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.condition', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'claim.context.disease', 'evidence_refs.gene_set_hash', 'context_evaluated', 'context_method', 'context_status', 'context_reason']

[report] contract view (head):
[SANITY][WARN] missing columns: ['schema_version']
  benchmark_id                run_id method cance

In [125]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/context_swap/gate_hard/tau_0.20


BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/context_swap/gate_hard/tau_0.20
report   True 78927
audit    True 60423
modules  True 14057
edges    True 88967
terms    True 3188
distilled True 40803
risk     True 913
card     True 627
meta     True 5619

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (40, 63)
[report] decisions: {'PASS': 40}
[report] tau unique: [0.2]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'condition', 'disease', 'cancer', 'comparison', 'tau', 'species', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.condition', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'claim.context.disease', 'evidence_refs.gene_set_hash', 'context_evaluated', 'context_method', 'context_status', 'context_reason']

[report] contract view (head):
[SANITY][WARN] missing columns: ['schema_version']
  benchmark_id                run_id meth

In [137]:
!ls /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80

audit_log.tsv		    llm_claims.prompt.json  run_meta.json
claims.proposed.tsv	    llm_claims.raw.json     run_meta.runner.json
distilled.tsv		    manifest.json	    sample_card.resolved.json
distilled.with_modules.tsv  modules.tsv		    sample_card.tau.json
evidence.normalized.tsv     report.jsonl	    term_gene_edges.tsv
llm_claims.candidates.json  report.md		    term_modules.tsv
llm_claims.meta.json	    risk_coverage.tsv


In [126]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80

BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80
report   True 78837
audit    True 60668
modules  True 14057
edges    True 88967
terms    True 3188
distilled True 40803
risk     True 913
card     True 563
meta     True 5506

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (40, 63)
[report] decisions: {'PASS': 35, 'ABSTAIN': 5}
[report] tau unique: [0.8]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'condition', 'disease', 'cancer', 'comparison', 'tau', 'species', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.condition', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'claim.context.disease', 'evidence_refs.gene_set_hash', 'context_evaluated', 'context_method', 'context_status', 'context_reason']

[report] contract view (head):
[SANITY][WARN] missing columns: ['schema_version']
  benchmark_id                run_i

In [127]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/context_swap/gate_hard/tau_0.80


BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/context_swap/gate_hard/tau_0.80
report   True 79157
audit    True 60668
modules  True 14057
edges    True 88967
terms    True 3188
distilled True 40803
risk     True 913
card     True 627
meta     True 5617

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (40, 63)
[report] decisions: {'PASS': 35, 'ABSTAIN': 5}
[report] tau unique: [0.8]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'condition', 'disease', 'cancer', 'comparison', 'tau', 'species', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.condition', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'claim.context.disease', 'evidence_refs.gene_set_hash', 'context_evaluated', 'context_method', 'context_status', 'context_reason']

[report] contract view (head):
[SANITY][WARN] missing columns: ['schema_version']
  benchmark_id             

In [130]:
!python paper/scripts/run_fig2_pipeline.py \
  --cancers HNSC \
  --variants ours,stress \
  --gate-modes hard \
  --taus 0.8 \
  --k-claims 100 \
  --stress-evidence-dropout-p 0.35 \
  --stress-evidence-dropout-min-keep 1 \
  --stress-contradictory-p 0.0 \
  --force

[run_fig2] plan
  benchmark_id: PANCAN_TP53_v1
  cancers: HNSC
  variants: ours,stress
  gate_modes: hard
  taus: 0.80
  k_claims: 100
  seed: 42
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out
  stress:
    recipe: dropout
    evidence_dropout_p: 0.350
    evidence_dropout_min_keep: 1
    contradictory_p: 0.000
    contradictory_max_extra: 0
[run_fig2] jobs: 2
[run_fig2] (1/2) HNSC ours gate=hard tau=0.80
[run_fig2] (2/2) HNSC stress gate=hard tau=0.80
[run_fig2] WARNING: stress variant ran but pipeline recorded both evidence_dropout_p=0 and contradictory_p=0 (stress may not have been applied). outdir=/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/stress/gate_hard/tau_0.80
[run_fig2] OK
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out


In [131]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80


BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80
report   True 78837
audit    True 60668
modules  True 14057
edges    True 88967
terms    True 3188
distilled True 40803
risk     True 913
card     True 563
meta     True 5507

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (40, 63)
[report] decisions: {'PASS': 35, 'ABSTAIN': 5}
[report] tau unique: [0.8]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'condition', 'disease', 'cancer', 'comparison', 'tau', 'species', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.condition', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'claim.context.disease', 'evidence_refs.gene_set_hash', 'context_evaluated', 'context_method', 'context_status', 'context_reason']

[report] contract view (head):
[SANITY][WARN] missing columns: ['schema_version']
  benchmark_id                run_i

In [132]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/stress/gate_hard/tau_0.80


BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/stress/gate_hard/tau_0.80
report   True 90572
audit    True 68674
modules  True 14057
edges    True 88967
terms    True 3188
distilled True 40803
risk     True 999
card     True 595
meta     True 5809

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (40, 66)
[report] decisions: {'FAIL': 35, 'ABSTAIN': 5}
[report] tau unique: [0.8]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'condition', 'disease', 'cancer', 'comparison', 'tau', 'species', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.condition', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'claim.context.disease', 'evidence_refs.gene_set_hash', 'context_evaluated', 'context_method', 'context_status', 'context_reason']

[report] contract view (head):
[SANITY][WARN] missing columns: ['schema_version']
  benchmark_id                run

In [133]:
!LLMPATH_BACKEND=ollama LLMPATH_CLAIM_MODE=llm python paper/scripts/run_fig2_pipeline.py \
  --cancers HNSC \
  --variants ours,stress \
  --gate-modes hard \
  --taus 0.8 \
  --k-claims 3 \
  --stress-evidence-dropout-p 0.35 \
  --stress-evidence-dropout-min-keep 1 \
  --stress-contradictory-p 0.0 \
  --context-review-mode llm \
  --force


[run_fig2] plan
  benchmark_id: PANCAN_TP53_v1
  cancers: HNSC
  variants: ours,stress
  gate_modes: hard
  taus: 0.80
  k_claims: 3
  seed: 42
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out
  stress:
    recipe: dropout
    evidence_dropout_p: 0.350
    evidence_dropout_min_keep: 1
    contradictory_p: 0.000
    contradictory_max_extra: 0
[run_fig2] jobs: 2
[run_fig2] (1/2) HNSC ours gate=hard tau=0.80
[run_fig2] report.jsonl is empty: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/report.jsonl


In [134]:
OUT="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/stress/gate_hard/tau_0.80"

!ls -lh $OUT/{run_meta.json,claims.proposed.tsv,audit_log.tsv,report.jsonl} 2>/dev/null || true
!wc -l  $OUT/claims.proposed.tsv  $OUT/audit_log.tsv  $OUT/report.jsonl 2>/dev/null || true

import json, os
p=os.environ.get("OUT","/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80")+"/run_meta.json"
m=json.load(open(p))
print("step:", m.get("step"), "status:", m.get("status"))
print("llm:", m.get("inputs",{}).get("llm",{}))
print("claims:", m.get("inputs",{}).get("claims",{}))

    41 /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/stress/gate_hard/tau_0.80/claims.proposed.tsv
    41 /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/stress/gate_hard/tau_0.80/audit_log.tsv
    40 /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/stress/gate_hard/tau_0.80/report.jsonl
   122 total
step: report_jsonl status: ok
llm: {'claim_mode_env': 'llm', 'backend_env': 'ollama', 'backend_enabled': True, 'backend_notes': ''}
claims: {'k_source': 'cfg', 'k_cfg': 3, 'k_env': '', 'k_card': '', 'k_effective': 3, 'k_env_raw': '', 'n_proposed_rows': 0, 'n_proposed_cols': 13, 'claim_payload_col': 'claim_json', 'claim_json_present': True, 'context_gate_mode': 'hard', 'context_review_mode': 'llm', 'context_review_runtime': {'evaluated': True, 'mode': 'proxy', 'gate_mode': 'hard', 'n_eval': 0, 'n_pass': 0, 'n_fail': 0, 'note': 'llm context review not implemented in pipeline; used proxy fallback', 'backend_enabled': True, 'seed': 42}}


In [135]:
import pandas as pd
p="paper/source_data/PANCAN_TP53_v1/out/HNSC/stress/gate_hard/tau_0.80/audit_log.tsv"
df=pd.read_csv(p, sep="\t")
print("cols:", [c for c in df.columns if "claim_json" in c])
print("has_claim_json:", "claim_json" in df.columns)
print("has_claim_json_str:", "claim_json_str" in df.columns)

cols: ['claim_json']
has_claim_json: True
has_claim_json_str: False


In [136]:
# context未評価でABSTAINになっているか
!grep -n '"decision_status": "ABSTAIN"' \
  paper/source_data/PANCAN_TP53_v1/out/HNSC/stress/gate_hard/tau_0.80/report.jsonl | head

# audit_notes に context_missing(hard) が残っているか
!grep -n "context_missing(hard)" \
  paper/source_data/PANCAN_TP53_v1/out/HNSC/stress/gate_hard/tau_0.80/report.jsonl | head


11:{"created_at": "2026-01-22T23:52:45.664623+00:00", "run_id": "1769125965_f2fc239190", "method": "ours", "condition": "HNSC", "disease": "HNSC", "cancer": "HNSC", "comparison": "TP53_mut_vs_TP53_wt", "tau": 0.8, "species": "", "claim": {"claim_id": "c_715e2eaa872c", "entity": "HALLMARK_INTERFERON_GAMMA_RESPONSE", "direction": "down", "context_keys": ["condition", "tissue", "perturbation", "comparison"], "context_evaluated": false, "context_method": "none", "context_status": null, "context_reason": null, "context_notes": null, "evidence_ref": {"module_id": "Mb6f784110271", "gene_ids": ["969", "4599", "3665", "567", "10561", "3663", "5698", "3937", "8743", "5770"], "term_ids": ["fgsea_msigdb_H_entrez:HALLMARK_INTERFERON_GAMMA_RESPONSE"], "gene_set_hash": "645976544bb2"}}, "decision_obj": {"status": "ABSTAIN", "reason": "unstable", "details": {"audit_notes": "survival[module]=0.766 < tau=0.80", "tau_used": 0.8, "term_survival_agg": 0.765625, "stability_scope": "module", "module_id_effec

In [77]:
for c in ("HNSC", "LUAD", "LUSC", "BRCA", "OV",  "UCEC",  "SKCM"):
    !Rscript paper/scripts/fig2_fgsea_to_evidence_table.R $c

[fgsea_to_evidence] inputs
  cancer: HNSC 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  out : /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  msigdbr_version: 25.1.1 
  fgsea_version  : 1.36.2 
[fgsea_to_evidence] stats diagnostics
  stats_len: 20262 
  stats_type: double 
  stats_na: 0 
  stats_range: -12.6721379948078,10.5253182402821 
  gene_example: 4060,6817,5153,1902,19594,9055,6447,16500,1970,8301 
  gene_id_type: entrez
[fgsea_to_evidence] msigdbr entrez column: ncbi_gene 
[fgsea_to_evidence] overlap diagnostics
  overlap_summary: min 24 p25 80 median 141 p75 165 max 193 
  pathways_with_overlap>=15: 50 / 50 
[fgsea_to_evidence] fgsea diagnostics
  n_pathways: 50 
  res_rows  : 50 
[fgsea_to_evidence] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  n_terms: 50 
[fgsea_to_evidence] inputs
  cancer: LUAD 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived

In [142]:
!python paper/scripts/run_fig2_pipeline.py \
  --cancers HNSC,LUAD,LUSC,BRCA,OV,UCEC,SKCM \
  --variants ours,shuffled_context \
  --gate-modes hard \
  --taus 0.2,0.4,0.6,0.8,0.9 \
  --force

[run_fig2] plan
  benchmark_id: PANCAN_TP53_v1
  cancers: HNSC,LUAD,LUSC,BRCA,OV,UCEC,SKCM
  variants: ours,shuffled_context
  gate_modes: hard
  taus: 0.20,0.40,0.60,0.80,0.90
  k_claims: 100
  seed: 42
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out
[run_fig2] jobs: 70
[run_fig2] (1/70) HNSC ours gate=hard tau=0.20
[run_fig2] (2/70) HNSC shuffled_context gate=hard tau=0.20
[run_fig2] (3/70) HNSC ours gate=hard tau=0.40
[run_fig2] (4/70) HNSC shuffled_context gate=hard tau=0.40
[run_fig2] (5/70) HNSC ours gate=hard tau=0.60
[run_fig2] (6/70) HNSC shuffled_context gate=hard tau=0.60
[run_fig2] (7/70) HNSC ours gate=hard tau=0.80
[run_fig2] (8/70) HNSC shuffled_context gate=hard tau=0.80
[run_fig2] (9/70) HNSC ours gate=hard tau=0.90
[run_fig2] (10/70) HNSC shuffled_context gate=hard tau=0.90
[run_fig2] (11/70) LUAD ours gate=hard tau=0.20
[run_fig2] (12/70) LUAD shuffled_context gate=hard tau=0.20
[run_fig2] (13/70) LUAD ours gate=hard tau=0.40
[run_fig2] (14/70) LUAD shuffled_c

In [143]:
!python paper/scripts/fig2_make_labels_template.py \
  --root paper/source_data/PANCAN_TP53_v1/out \
  --out  paper/source_data/PANCAN_TP53_v1/labels.tsv \
  --per-run 20 \
  --prefer-status PASS \
  --with-hints

[OK] wrote: paper/source_data/PANCAN_TP53_v1/labels.tsv rows=150
[NEXT] Fill human_label with: ACCEPT / REJECT / SHOULD_ABSTAIN (partial labels ok)


In [141]:
!python paper/scripts/fig2_collect_risk_coverage.py \
  --root /work/paper/source_data/PANCAN_TP53_v1/out \
  --out  /work/paper/source_data/PANCAN_TP53_v1/out/fig2_risk_coverage.tsv \
  --benchmark-id PANCAN_TP53_v1


[OK] wrote: /work/paper/source_data/PANCAN_TP53_v1/out/fig2_risk_coverage.tsv rows=70
[WARN] n_pass_labeled is 0 for all rows. Use risk_proxy or provide --labels.


In [144]:
!python paper/scripts/bench_fig2.py \
  --reports paper/source_data/PANCAN_TP53_v1/out \
  --labels  paper/source_data/PANCAN_TP53_v1/labels.tsv \
  --out     /work/paper/source_data/PANCAN_TP53_v1/out/fig2_risk_coverage.tsv \
  --taus    0:1:0.05 \
  --rater-id R1


Traceback (most recent call last):
  File "/work/paper/scripts/bench_fig2.py", line 315, in <module>
    main()
  File "/work/paper/scripts/bench_fig2.py", line 303, in main
    reports_df = load_reports(report_paths)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/work/paper/scripts/bench_fig2.py", line 123, in load_reports
    raise ValueError(
ValueError: Missing metrics.survival in paper/source_data/PANCAN_TP53_v1/out/BRCA/ours/gate_hard/tau_0.20/report.jsonl line 1 (claim_id=c_a1ed09ef8099)


In [104]:
!python paper/scripts/fig2_plot_multipanel.py \
  --in  /work/paper/source_data/PANCAN_TP53_v1/out/fig2_risk_coverage.tsv \
  --out paper/source_data/PANCAN_TP53_v1/fig/Fig2_Supp_multipanel_proxy.pdf \
  --ycol risk_proxy \
  --benchmark-id PANCAN_TP53_v1 \
  --cols 4 \
  --fontsize 16 \
  --title "Risk–coverage curves across cancers"
